Chapter 4: Mining data streams

A data stream is abstractly an infitite sequence of data points: x1, x2, ....

We have lots of data.

Determine if an item $a$ is in S, when we do not have much space to store all items.  Additionally, we want to be able to answer this membership question very quickly.

Solution: Bloom filter.  What is a Bloom filter?

- It is very fast.
- It is probabilistic.
- It can make false positive mistakes.  But it makes no false negative mistakes.

What is a False Positive?

Let's say you are making a prediction.  A Positive is essentially a "Yes" answer.  A Negative is a "No" answer.  A True Positive is a Yes answer that is correct.  A False Positive is a Yes answer that is incorrect.

Example: 

- "have we seen this ip address before?"  If the answer is Yes for x, but it is wrong. Then we have a false positive.

- "is this a spam email?"  If the answer is Yes for x, but it is wrong, then we have a false positive.


What is a Bloom filter?

- A set S of m items that we want to store. These can come from a stream of data.
- An array of n bits, initially all 0's.
- k random hash functions: $h_1, h_2, ..., h_k$
- Insertion of $x$ into the Bloom filter: given an item $x \in S$, $h(x)$ maps $x$ to one of the n bits.  Suppose $h(x) = i$.  Then we set bit $i$ to 1.


In [3]:
n = 20
filter = [0]*n

def hash(x):
    return (5*x + 3) % n

i = hash(150)
filter[i] = 1
print(filter)

y = 325
j = hash(y)
print(j, filter[j])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
8 0


Construction of the Bloom filter: for each item $x \in S$, set $h_i(x)$ to 1 for $1 \le i \le k$.

Querying: to determine if an item y is in S, we look at $h_1(y), h_2(y), \cdots, h_k(y)$, if all bits are 1's, return Yes.

How do we find hash functions?




In [4]:
import random

n = 100

def random_hash_function(n):
    a = random.randint(1, n)
    b = random.randint(1, n)
    def f(x):
        print('ab',a,b)
        return (a*x + b)%n
    return f

h1 = random_hash_function(n)
h2 = random_hash_function(n)

for i in range(0,5):
    print(i, h1(i),h2(i))



ab 62 38
ab 86 10
0 38 10
ab 62 38
ab 86 10
1 0 96
ab 62 38
ab 86 10
2 62 82
ab 62 38
ab 86 10
3 24 68
ab 62 38
ab 86 10
4 86 54


What is the probability of false positives?

+ $P(h_1(x) == 1) = {1 \over n}$

+ When x is inserted with h1, what is the probability that bit i is 0?

$P(h_1(x) == 0) = (1 - {1 \over n})$


+ When x is inserted with k hash functions, what is the probability that bit i is 0?

$P(h_i(x) == 0, 1 \le i \le k) = (1 - {1 \over n})^k$



+ When all m items are inserted with k hash functions, what is the probability that bit i is 0?

$P(h_i(x_j) == 0, 1 \le i \le k, 1 \le j \le m) = (1 - {1 \over n})^{km}$


+ When all m items are inserted with k hash functions, what is the probability that bit i is 1?

$1 - (1 - {1 \over n})^{km}$



+ Given m items have been inserted, what is the probability that an item $y \notin S$, $h_1(y)$ equal to 1?

$1 - (1 - {1 \over n})^{km}$


+ What is the probability that for $y \notin S$, $h_1(y), h_2(y), \cdots, h_k(y)$ equal to 1?

$(1 - (1 - {1 \over n})^{km})^k$


Approximation, $1-x \approx e^{-x}$, for small values of x.

$(1 - (1 - {1 \over n})^{km})^k \approx (1 - e^{-km \over n})^k$






In [3]:

def false_positive_prob(n,m,k):
    a = (1-1/n)**(k*m)
    b = (1-a)**k
    return b


n, m, k = 10000, 1000, 5
print(n, m, k, false_positive_prob(n,m,k))



10000 1000 5 0.009432746679838469
